In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
#ee.Authenticate()
#geemap.update_package()

ee.Initialize()

Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

##Adding every plot coordinate
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'
fc_all = geemap.csv_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

In [3]:

##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.
def maskL5sr(image):
  ## Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  ##Get the pixel QA band.
  qa = image.select('pixel_qa')

  ##Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  ##Return the masked image, scaled to reflectance, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])

In [4]:

def addFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B6'),
            'RED': image.select('B4'),
            'GREEN': image.select('B3')
        })
    
    return image.addBands(flats.rename('flats'))

##Notes: This is setup for Landsat 8 - apply to l5?

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B5'),
            'RED': image.select('B3'),
            'GREEN': image.select('B2')
        })
    
    return image.addBands(flats.rename('flats'))

##MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.1) #less than or equal to 0.1 - change?
    return image.updateMask(mask1)

##ADDING NDVI (for min/max variables)
def addL5ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('B4'),
            'RED': image.select('B3'),
            'GREEN': image.select('B2')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

def addL8ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('B5'),
            'RED': image.select('B4'),
            'GREEN': image.select('B3')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

In [5]:
##Pixel extraction functions - addDate for dateless images/collections
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

##For Landsat images:
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 30 
    )
    return feature

##FOR 10m DEM:
def demExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 10 
    )
    return feature

##FOR 1m DEM:
def dem1Extraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 1 
    )
    return feature

In [6]:

##Adding DEM
dem = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m.
dem1 = ee.ImageCollection('USGS/3DEP/1m')

##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, dem_params, '10m DEM')
Map.addLayer(dem1, dem_params, '1m DEM')

In [7]:
##Calculating mean pixel values for time periods within each year

def monthly_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

## Compute monthly averages
# monthly_sowing_Avg = monthly_Avg (ndvi_sowSeason, years, sowingMonths)

##Months and years are lists

years_ls5 = range(2000, 2012)
years_ls7 = range(2012, 2013)
years_ls8 = range(2013, 2021)
years_dm = range(2000, 2021)

months = range(5,11)
months_daymet = range(1,12)

ls5_collect = ee.ImageCollection(
    'LANDSAT/LT05/C01/T1_SR'
).filterBounds(fc_all).map(maskL5sr).map(addFLATSL5).map(maskFLATS).map(addL5ndvi) ##MIGRATE TO COLLECTION 2 DATA

ls7_collect = ee.ImageCollection(
    'LANDSAT/LE07/C01/T1_SR'
).filterBounds(fc_all).map(maskL5sr).map(addFLATSL5).map(maskFLATS).map(addL5ndvi) ####SHOULD BE maskFLATSL5 - check BANDS!!!

##MIGRATE TO COLLECTION 2 DATA

ls8_collect = ee.ImageCollection(
    'LANDSAT/LC08/C01/T1_SR'
).filterBounds(fc_all).map(maskL5sr).map(addFLATS).map(maskFLATS).map(addL8ndvi) ##NEW L8 DATASET: "LANDSAT/LC08/C02/T1_L2"

monthly_ls5 = monthly_Avg(ls5_collect, years = years_ls5)
monthly_ls7 = monthly_Avg(ls7_collect, years = years_ls7)
monthly_ls8 = monthly_Avg(ls8_collect, years = years_ls8)

monthly_ls5.size().getInfo()

12

In [8]:
list_5 = monthly_ls5.toList(monthly_ls5.size())
list_7 = monthly_ls7.toList(monthly_ls7.size())
list_8 = monthly_ls8.toList(monthly_ls8.size())

# ls5_2000 = ee.Image(list_5.get(0))
# ls5_2001 = ee.Image(list_5.get(1))
# ls5_2002 = ee.Image(list_5.get(2))
# ls5_2004 = ee.Image(list_5.get(4)) ##SKIP 2003
# ls5_2005 = ee.Image(list_5.get(5))
# ls5_2006 = ee.Image(list_5.get(6))
# ls5_2007 = ee.Image(list_5.get(7))
# ls5_2008 = ee.Image(list_5.get(8))
# ls5_2009 = ee.Image(list_5.get(9))
# ls5_2010 = ee.Image(list_5.get(10))
# ls5_2011 = ee.Image(list_5.get(11))

# ls7_2012 = ee.Image(list_7.get(0))

# ls8_2013 = ee.Image(list_8.get(0))
# ls8_2014 = ee.Image(list_8.get(1))
# ls8_2015 = ee.Image(list_8.get(2))
# ls8_2016 = ee.Image(list_8.get(3))
# ls8_2017 = ee.Image(list_8.get(4))
# ls8_2018 = ee.Image(list_8.get(5))
# ls8_2019 = ee.Image(list_8.get(6))
# ls8_2020 = ee.Image(list_8.get(7))

# # ls5_2004.getInfo()

In [9]:
#With loop to cut out unnecessary code:
years_l5 = range(2000, 2012)
years_l7 = range(2012, 2013)
years_l8 = range(2013, 2021)

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(list_5.get(i))))
    sample5 = ls5_x[ls5_x['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(list_7.get(i))))
    sample7 = ls7_x[ls7_x['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(list_8.get(i))))
    sample8 = ls8_x[ls8_x['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)

landsat_extract = pd.concat([l5_extract,l7_extract,l8_extract])

landsat_extract

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,...,B3,B4,B5,B6,B7,flats,ndvi,B1,B10,B11
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,...,0.059200,0.115744,0.101289,0.294367,0.064078,0.007205,0.326278,0.050189,NaN,NaN
19,2,A1,2000,1,1,31.536199,1152.400,21,-81.296118,10/11/2000,...,0.059557,0.133157,0.098757,0.295686,0.055271,0.051133,0.381316,0.046829,NaN,NaN
21,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,...,0.061575,0.124300,0.093700,0.297550,0.049700,0.026039,0.337292,0.047075,NaN,NaN
33,4,A1,2000,1,1,31.451676,1534.304,25,-81.365498,10/10/2000,...,0.056500,0.103800,0.083533,0.295067,0.049000,0.005872,0.295413,0.043333,NaN,NaN
47,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,...,0.065087,0.162000,0.130325,0.295213,0.072713,0.016011,0.429154,0.048300,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2966,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,...,0.048275,0.044600,0.166175,0.093975,0.046150,0.010564,0.568241,0.012075,0.295675,0.292600
2972,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,...,0.046325,0.043050,0.145075,0.077950,0.038850,0.018787,0.533996,0.013100,0.296000,0.293025
2976,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,...,0.045133,0.044133,0.116500,0.061933,0.032367,0.019022,0.435850,0.015133,0.295100,0.292367
2980,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,...,0.045133,0.044133,0.116500,0.061933,0.032367,0.019022,0.435850,0.015133,0.295100,0.292367


In [10]:
vis_param = {'min': 0, 
             'max': 0.2, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

x= ee.Image(list_5.get(0))
y= ee.Image(list_8.get(0))

Map.addLayer(x, vis_param)
Map.addLayer(y, vis_param)

Map.addLayer(fc_all)
Map

Map(center=[31.539096, -81.422318], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

In [11]:
#Daymet for loop and elevation

##Daymet

def dm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterBounds(fc_all)

monthly_dm = dm_Avg(daymet, years = years_dm)

dm_list = monthly_dm.toList(monthly_dm.size())

years_dm = range(2000, 2021)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(dm_list.get(i))))
    sampledm = dm_x[dm_x['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
daymet_extract = pd.concat(daymet_list)

daymet_extract

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,swe,tmax,srad,tmin,vp,prcp,dayl
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,0,25.069397,343.573212,14.079397,1765.701416,2.489973,43200.085938
19,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,0,24.539864,336.004730,15.079507,1874.970703,2.319699,43200.085938
31,4,A1,2000,1,1,31.451676,1534.304,25,-81.365498,10/10/2000,0,24.868795,340.215027,14.635425,1827.156250,2.472329,43200.085938
45,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,0,24.818274,340.010864,14.757424,1839.845947,2.445726,43200.085938
64,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,0,24.615616,334.624725,15.242548,1893.765625,2.393671,43200.085938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2875,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2879,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2883,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938


In [12]:


# dm_2000 = ee.Image(dm_list.get(0))
# dm_2001 = ee.Image(dm_list.get(1))
# dm_2002 = ee.Image(dm_list.get(2))
# dm_2004 = ee.Image(dm_list.get(4)) ##SKIP 2003!
# dm_2005 = ee.Image(dm_list.get(5))
# dm_2006 = ee.Image(dm_list.get(6))
# dm_2007 = ee.Image(dm_list.get(7))
# dm_2008 = ee.Image(dm_list.get(8))
# dm_2009 = ee.Image(dm_list.get(9))
# dm_2010 = ee.Image(dm_list.get(10))
# dm_2011 = ee.Image(dm_list.get(11))
# dm_2012 = ee.Image(dm_list.get(12))
# dm_2013 = ee.Image(dm_list.get(13))
# dm_2014 = ee.Image(dm_list.get(14))
# dm_2015 = ee.Image(dm_list.get(15))
# dm_2016 = ee.Image(dm_list.get(16))
# dm_2017 = ee.Image(dm_list.get(17))
# dm_2018 = ee.Image(dm_list.get(18))
# dm_2019 = ee.Image(dm_list.get(19))
# dm_2020 = ee.Image(dm_list.get(20))

# # dm_2020.getInfo()

In [13]:
dem_vals = geemap.ee_to_pandas(demExtraction(dem)) ##10m dataset
# dem_vals = geemap.ee_to_pandas(dem1.map(dem1Extraction).flatten()) ##ONE METER DATASET

# # daymet_vals = geemap.ee_to_pandas(monthly_dm.map(rasterExtraction).flatten())
# xdm_2000 = geemap.ee_to_pandas(rasterExtraction(dm_2000))
# xdm_2001 = geemap.ee_to_pandas(rasterExtraction(dm_2001))
# xdm_2002 = geemap.ee_to_pandas(rasterExtraction(dm_2002))
# xdm_2004 = geemap.ee_to_pandas(rasterExtraction(dm_2004)) ##SKIP 2003
# xdm_2005 = geemap.ee_to_pandas(rasterExtraction(dm_2005))
# xdm_2006 = geemap.ee_to_pandas(rasterExtraction(dm_2006))
# xdm_2007 = geemap.ee_to_pandas(rasterExtraction(dm_2007))
# xdm_2008 = geemap.ee_to_pandas(rasterExtraction(dm_2008))
# xdm_2009 = geemap.ee_to_pandas(rasterExtraction(dm_2009))
# xdm_2010 = geemap.ee_to_pandas(rasterExtraction(dm_2010))
# xdm_2011 = geemap.ee_to_pandas(rasterExtraction(dm_2011))
# xdm_2012 = geemap.ee_to_pandas(rasterExtraction(dm_2012))
# xdm_2013 = geemap.ee_to_pandas(rasterExtraction(dm_2013))
# xdm_2014 = geemap.ee_to_pandas(rasterExtraction(dm_2014))
# xdm_2015 = geemap.ee_to_pandas(rasterExtraction(dm_2015))
# xdm_2016 = geemap.ee_to_pandas(rasterExtraction(dm_2016))
# xdm_2017 = geemap.ee_to_pandas(rasterExtraction(dm_2017))
# xdm_2018 = geemap.ee_to_pandas(rasterExtraction(dm_2018))
# xdm_2019 = geemap.ee_to_pandas(rasterExtraction(dm_2019))
# xdm_2020 = geemap.ee_to_pandas(rasterExtraction(dm_2020))

# ##Raster vals
# # geemap.ee_to_pandas(monthly_ls5.map(rasterExtraction).flatten())
# x_2000 = geemap.ee_to_pandas(rasterExtraction(ls5_2000))
# x_2001 = geemap.ee_to_pandas(rasterExtraction(ls5_2001))
# x_2002 = geemap.ee_to_pandas(rasterExtraction(ls5_2002))
# x_2004 = geemap.ee_to_pandas(rasterExtraction(ls5_2004)) ##SKIP 2003
# x_2005 = geemap.ee_to_pandas(rasterExtraction(ls5_2005))
# x_2006 = geemap.ee_to_pandas(rasterExtraction(ls5_2006))
# x_2007 = geemap.ee_to_pandas(rasterExtraction(ls5_2007))
# x_2008 = geemap.ee_to_pandas(rasterExtraction(ls5_2008))
# x_2009 = geemap.ee_to_pandas(rasterExtraction(ls5_2009))
# x_2010 = geemap.ee_to_pandas(rasterExtraction(ls5_2010))
# x_2011 = geemap.ee_to_pandas(rasterExtraction(ls5_2011))
# x_2012 = geemap.ee_to_pandas(rasterExtraction(ls7_2012))
# x_2013 = geemap.ee_to_pandas(rasterExtraction(ls8_2013))
# x_2014 = geemap.ee_to_pandas(rasterExtraction(ls8_2014))
# x_2015 = geemap.ee_to_pandas(rasterExtraction(ls8_2015))
# x_2016 = geemap.ee_to_pandas(rasterExtraction(ls8_2016))
# x_2017 = geemap.ee_to_pandas(rasterExtraction(ls8_2017))
# x_2018 = geemap.ee_to_pandas(rasterExtraction(ls8_2018))
# x_2019 = geemap.ee_to_pandas(rasterExtraction(ls8_2019))
# x_2020 = geemap.ee_to_pandas(rasterExtraction(ls8_2020))


In [14]:
##Subsetting dataframes for each year
# x_2000 = x_2000[x_2000['Year'] == 2000]
# x_2001 = x_2001[x_2001['Year'] == 2001]
# x_2002 = x_2002[x_2002['Year'] == 2002]
# x_2004 = x_2004[x_2004['Year'] == 2004] ##SKIP 2003
# x_2005 = x_2005[x_2005['Year'] == 2005]
# x_2006 = x_2006[x_2006['Year'] == 2006]
# x_2007 = x_2007[x_2007['Year'] == 2007]
# x_2008 = x_2008[x_2008['Year'] == 2008]
# x_2009 = x_2009[x_2009['Year'] == 2009]
# x_2010 = x_2010[x_2010['Year'] == 2010]
# x_2011 = x_2011[x_2011['Year'] == 2011]
# x_2012 = x_2012[x_2012['Year'] == 2012]
# x_2013 = x_2013[x_2013['Year'] == 2013]
# x_2014 = x_2014[x_2014['Year'] == 2014]
# x_2015 = x_2015[x_2015['Year'] == 2015]
# x_2016 = x_2016[x_2016['Year'] == 2016]
# x_2017 = x_2017[x_2017['Year'] == 2017]
# x_2018 = x_2018[x_2018['Year'] == 2018]
# x_2019 = x_2019[x_2019['Year'] == 2019]
# x_2020 = x_2020[x_2020['Year'] == 2020]

# landsat_list = [
#     x_2000,x_2001,x_2002,x_2004,x_2005,x_2006,x_2007,x_2008,x_2009,x_2010,x_2011,x_2012,x_2013,x_2014,x_2015,x_2016,x_2017,
#     x_2018,x_2019,x_2020
# ]

# landsat_extract = pd.concat(landsat_list)

# xdm_2000 = xdm_2000[xdm_2000['Year'] == 2000]
# xdm_2001 = xdm_2001[xdm_2001['Year'] == 2001]
# xdm_2002 = xdm_2002[xdm_2002['Year'] == 2002]
# xdm_2004 = xdm_2004[xdm_2004['Year'] == 2004] ##SKIP 2003
# xdm_2005 = xdm_2005[xdm_2005['Year'] == 2005]
# xdm_2006 = xdm_2006[xdm_2006['Year'] == 2006]
# xdm_2007 = xdm_2007[xdm_2007['Year'] == 2007]
# xdm_2008 = xdm_2008[xdm_2008['Year'] == 2008]
# xdm_2009 = xdm_2009[xdm_2009['Year'] == 2009]
# xdm_2010 = xdm_2010[xdm_2010['Year'] == 2010]
# xdm_2011 = xdm_2011[xdm_2011['Year'] == 2011]
# xdm_2012 = xdm_2012[xdm_2012['Year'] == 2012]
# xdm_2013 = xdm_2013[xdm_2013['Year'] == 2013]
# xdm_2014 = xdm_2014[xdm_2014['Year'] == 2014]
# xdm_2015 = xdm_2015[xdm_2015['Year'] == 2015]
# xdm_2016 = xdm_2016[xdm_2016['Year'] == 2016]
# xdm_2017 = xdm_2017[xdm_2017['Year'] == 2017]
# xdm_2018 = xdm_2018[xdm_2018['Year'] == 2018]
# xdm_2019 = xdm_2019[xdm_2019['Year'] == 2019]
# xdm_2020 = xdm_2020[xdm_2020['Year'] == 2020]

# daymet_list = [
#     xdm_2000,xdm_2001,xdm_2002,xdm_2004,xdm_2005,xdm_2006,xdm_2007,xdm_2008,xdm_2009,xdm_2010,xdm_2011,xdm_2012,xdm_2013,
#     xdm_2014,xdm_2015,xdm_2016,xdm_2017,xdm_2018,xdm_2019,xdm_2020
# ]

# daymet_extract = pd.concat(daymet_list)

dfx = pd.merge(landsat_extract, dem_vals, how = 'left')

df = pd.merge(dfx, daymet_extract, how = 'right')

display(df)

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,...,B10,B11,elevation,swe,tmax,srad,tmin,vp,prcp,dayl
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,...,NaN,NaN,0.303575,0,25.069397,343.573212,14.079397,1765.701416,2.489973,43200.085938
1,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,...,NaN,NaN,1.106164,0,24.539864,336.004730,15.079507,1874.970703,2.319699,43200.085938
2,4,A1,2000,1,1,31.451676,1534.304,25,-81.365498,10/10/2000,...,NaN,NaN,0.879874,0,24.868795,340.215027,14.635425,1827.156250,2.472329,43200.085938
3,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,...,NaN,NaN,1.041272,0,24.818274,340.010864,14.757424,1839.845947,2.445726,43200.085938
4,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,...,NaN,NaN,0.945370,0,24.615616,334.624725,15.242548,1893.765625,2.393671,43200.085938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,...,0.295675,0.292600,1.277348,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2908,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,...,0.296000,0.293025,0.411499,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2909,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,...,0.295100,0.292367,0.302258,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938
2910,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,...,0.295100,0.292367,0.101274,0,26.537781,333.358398,15.792986,1939.959351,3.794055,43200.085938


In [15]:
##Bands and indices
df['Sensor'] = np.where(df['Year']<2013, 'Landsat 5', 'Landsat 8') ##make sure no other sensors are being used

df.loc[df['Year'] == 2012, 'Sensor'] = 'Landsat 7'

df['ndvi'] = np.where(df['Sensor'] == 'Landsat 8', (df['B5']-df['B4'])/(df['B5']+df['B4']), \
                      (df['B4']-df['B3'])/(df['B4']+df['B3'])) ##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band'] = np.where(df['Sensor'] == 'Landsat 8', df['B2'], df['B1'])
df['Green_band'] = np.where(df['Sensor'] == 'Landsat 8', df['B3'], df['B2'])
df['Red_band'] = np.where(df['Sensor'] == 'Landsat 8', df['B4'], df['B3'])
df['NIR_band'] = np.where(df['Sensor'] == 'Landsat 8', df['B5'], df['B4'])
df['SWIR1_band'] = np.where(df['Sensor'] == 'Landsat 8', df['B6'], df['B5'])
df['SWIR2_band'] = np.where(df['Sensor'] == 'Landsat 8', df['B7'], df['B7'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi'] = ((df['NIR_band']-df['Red_band'])*1.5)/(df['NIR_band']+df['Red_band']+0.5)
df['wdrvi5'] = (0.5*df['NIR_band']-df['Red_band'])/(0.5*df['NIR_band']+df['Red_band'])
df['nd_r_g'] = (df['Red_band']-df['Green_band'])/(df['Red_band']+df['Green_band'])
df['nd_g_b'] = (df['Green_band']-df['Blue_band'])/(df['Green_band']+df['Blue_band'])
df['nd_swir2_nir'] = (df['SWIR2_band']-df['NIR_band'])/(df['SWIR2_band']+df['NIR_band'])
df['nd_swir2_r'] = (df['SWIR2_band']-df['Red_band'])/(df['SWIR2_band']+df['Red_band'])

display(df)

##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'df_min.csv')
# df.to_csv(out_csv, index = False)

,Site,Species_Code,Year,Zone,Plot,Latitude,Plant_Biomass,Sample_Size,Longitude,Date,...,Red_band,NIR_band,SWIR1_band,SWIR2_band,savi,wdrvi5,nd_r_g,nd_g_b,nd_swir2_nir,nd_swir2_r
0,1,A1,2000,1,1,31.538627,516.324,23,-81.422539,10/11/2000,...,0.059200,0.115744,0.101289,0.064078,0.125665,-0.011342,-0.004019,0.086367,-0.287321,0.039567
1,3,A1,2000,1,1,31.518920,710.396,20,-81.228956,10/18/2000,...,0.061575,0.124300,0.093700,0.049700,0.137179,0.004647,-0.010048,0.143312,-0.428736,-0.106718
2,4,A1,2000,1,1,31.451676,1534.304,25,-81.365498,10/10/2000,...,0.056500,0.103800,0.083533,0.049000,0.107451,-0.042435,0.002662,0.129270,-0.358639,-0.071090
3,5,A1,2000,1,1,31.436090,2022.264,25,-81.339919,10/10/2000,...,0.065087,0.162000,0.130325,0.072713,0.199933,0.108924,-0.043358,0.190192,-0.380412,0.055334
4,6,A1,2000,1,1,31.388423,1032.632,20,-81.279792,10/12/2000,...,0.075356,0.141644,0.112500,0.066044,0.138680,-0.031012,-0.015103,0.113678,-0.364006,-0.065849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,7,A4,2019,2,8,31.334597,2365.620,14,-81.478039,10/8/2019,...,0.044600,0.166175,0.093975,0.046150,0.256569,0.301420,-0.039569,0.385719,-0.565289,0.017080
2908,7,A4,2019,1,12,31.335017,1821.010,12,-81.478711,10/8/2019,...,0.043050,0.145075,0.077950,0.038850,0.222398,0.255110,-0.036643,0.358504,-0.577545,-0.051282
2909,7,A4,2019,1,13,31.334934,2482.584,14,-81.478709,10/8/2019,...,0.044133,0.116500,0.061933,0.032367,0.164312,0.137881,-0.011202,0.329406,-0.565159,-0.153813
2910,7,A4,2019,1,16,31.334682,2511.936,17,-81.478564,10/8/2019,...,0.044133,0.116500,0.061933,0.032367,0.164312,0.137881,-0.011202,0.329406,-0.565159,-0.153813
